In [92]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [93]:
url = "https://www.amazon.eg/s?k=%D9%82%D8%B7%D8%B7&crid=1W9DGJBWD56SO&sprefix=%D9%82%D8%B7%2Caps%2C169&ref=nb_sb_noss_2"
response = requests.get(url, headers={
    'User-Agent': 'Mozilla 5 (Educational)'
})
print('contains listitem is ', response.text.count('listitem'))


contains listitem is  60


In [94]:
soup = BeautifulSoup(response.text, 'html.parser')
listitems = soup.find_all('div', attrs={
    'role':'listitem'
})
print(len(listitems))
print(listitems[0])

60
<div class="sg-col-4-of-4 sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-8 sg-col-4-of-20" data-asin="B0D62ZXDQB" data-component-type="s-search-result" data-index="2" data-uuid="bee1f7c7-7ca6-4948-bb99-ff189e9f45ff" id="bee1f7c7-7ca6-4948-bb99-ff189e9f45ff" role="listitem"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-2" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-csa-c-item-id="amzn1.asin.1.B0D62ZXDQB" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render=""><span class="a-declarative" data-action="puis-card-container-declarative" data-csa-c-func-deps="aui-da-puis-card-container-declarative" data-csa-c-item-id="amzn1.asin.B0D62ZXDQB:amzn1.promotion.A3H6NP5UAGBE1" data-csa-c-owner="puis" data-csa-c-posx="1" data-csa-c-type="item" data-render-id="r2mr2rkrnag7lm23hjep76l15xa" data-version-i

In [95]:
def clean_text(s):
    return re.sub(r'[\u200e\u200f\xa0]', ' ', s).strip()

In [96]:
items = []
for listitem in listitems:
    item = {}
    
    # Title
    title = listitem.find('div', attrs={
        'data-cy':'title-recipe'
    })
    if title:
        item['title'] = title.text
    
    # Price
    price = listitem.find('div', attrs={
        'data-cy':'price-recipe'
    })
    if(price):
        symbol = clean_text(price.select_one('span.a-price-symbol').text)
        whole = clean_text(price.select_one('span.a-price-whole').text)
        fraction = clean_text(price.select_one('span.a-price-fraction').text)
        # print('whole', whole)
        # print('fraction', fraction)
        item['price'] = float(f'{whole}{fraction}')
        item['currency'] = symbol
    
    # Badge (ex: Best seller)
    badge = listitem.select_one('div#amazonBadges-capsule-view div div div')
    if badge:
        # inside inside the badge (ex: id="BEST_SELLER" inside it) 
        # .. assuming one badge for each .. if more 1 is taken
        badgeParts = list(badge.children)
        badgeTitle = badgeParts[0].text
        badgeAmong = badgeParts[1].text
        item['badge'] = badgeTitle
        item['badge among'] = badgeAmong

    # Image
    img = listitem.find('div', attrs={
        'data-cy': 'image-container'
    })
    if(img):
        img = img.select_one('img')
    if(img):
        item['image'] = img.attrs['src']
    else:
        item['image'] = None
    # print(item.get('image'))

    items.append(item)
    print(item)  

{'title': 'فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، فرشاة بخار تنظيف ذاتي للتدليك، تزيل الشعر الطائر المتناثر، فرشاة إزالة الشعر لإزالة الشعر المتشابك والمتناثر، مشط إزالة الشعر للقطط والكلاب (أبيض) ', 'price': 114.0, 'currency': 'جنيه', 'badge': 'الأكثر مبيعاً', 'badge among': 'في Cat Brushes', 'image': 'https://m.media-amazon.com/images/I/61dopS6y4BL._AC_UL320_.jpg'}
{'title': 'لوح خدش للقطط والحيوانات الأليفة من الورق المقوى من سكاي-تاتش، وسادة خدش للقطط، لعبة خدش تفاعلية مقاس 43 × 21.5 × 3 سم ', 'price': 165.0, 'currency': 'جنيه', 'badge': 'الأكثر مبيعاً', 'badge among': 'في Cat Scratching Pads', 'image': 'https://m.media-amazon.com/images/I/71QUDPK1T5L._AC_UL320_.jpg'}
{'title': 'طعام رطب للقطط فريسكيز بقطع اللحم البقري في المرق من بورينا، 85 جرام ', 'price': 39.6, 'currency': 'جنيه', 'badge': 'الأكثر مبيعاً', 'badge among': 'في Cat Food Pouches', 'image': 'https://m.media-amazon.com/images/I/71yGsWtgcSL._AC_UL320_.jpg'}
{'title': 'التفاعلية LED التدريب مضحك القط اللعب لعبة مؤ

In [97]:
df = pd.DataFrame(items)

In [98]:
print(df)

                                                title   price currency  \
0   فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، ف...  114.00     جنيه   
1   لوح خدش للقطط والحيوانات الأليفة من الورق المق...  165.00     جنيه   
2   طعام رطب للقطط فريسكيز بقطع اللحم البقري في ال...   39.60     جنيه   
3   التفاعلية LED التدريب مضحك القط اللعب لعبة مؤش...   37.99     جنيه   
4   رمل فضلات القطط سريع التكتل ومضاد للبكتيريا وا...   85.00     جنيه   
5   طعام رطب للقطط بنكهة الدجاج من باوير، 400 جرام...   50.00     جنيه   
6   رمل متكتل لفضلات القطط من دودزي، 5 لتر (بنفسجي...  141.10     جنيه   
7   سجادة خدش للقطط - لوح خدش للقطط - وسادة مريحة ...  150.00     جنيه   
8   SKEIDO سرير كلب للقطط دافئ دافئ بيت الكلب النا...  199.00     جنيه   
9   اوعية طعام وشراب مزدوجة للقطط والكلاب والحيوان...  192.00     جنيه   
10  طعام مجفف بالدجاج في المرق للقطط فريسكيز من بي...   39.60     جنيه   
11  فرشاة العناية بالقطط من اي تي بلس، فرش سليكر ذ...   80.00     جنيه   
12  نافورة مياه للقطط والحيوانات الألي

In [99]:
df.describe(include='all')

,title,price,currency,badge,badge among,image
count,48,48.000000,48,10,10,48
unique,48,NaN,1,1,9,48
top,فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، ف...,NaN,جنيه,الأكثر مبيعاً,في Cat Food Pouches,https://m.media-amazon.com/images/I/61dopS6y4B...
freq,1,NaN,48,10,2,1
mean,NaN,158.929792,NaN,NaN,NaN,NaN
std,NaN,122.576352,NaN,NaN,NaN,NaN
min,NaN,37.990000,NaN,NaN,NaN,NaN
25%,NaN,79.750000,NaN,NaN,NaN,NaN
50%,NaN,117.050000,NaN,NaN,NaN,NaN
75%,NaN,208.497500,NaN,NaN,NaN,NaN
